In [2]:
import copy
import random
import math


In [24]:
L = 1 # Lower Bound of consecutive games. Set to 0 or 1 for normal TTP-k
U = 2 # Upper Bound of consecutive games. Also defined as k.

# Team distances such that the distance d(i,j) = d[i][j] for teams i and j.
# It stands that d[i][j] = d[j][i] and d[i][i] = 0.
d = [[0, 1, 4, 2],
    [1, 0, 5, 2],
    [4, 5, 0, 1],
    [2, 2, 1, 0]]

# 2(N-1) rounds => 6 rounds for 4 teams.
# Schedule of (i, j) where i is the team and j is the round. A positive number indicates home game, while a negative an away.
# The number itself indicates the opponent team starting from Team 1. 
# For example, in Day 3 for Team 1 (example_schedule[0][2]) they will play against Team 2 away from home.
example_schedule = [[2, 3, -2, -3, 4, -4],
                    [-1, -4, 1, 4, -3, 3],
                    [4, -1, -4, 1, 2, -2],
                    [-3, 2, 3, -2, -1, 1]]

#TODO: Delete this. Found in original TTP paper. Also delete all the above L, U, d and example.
complex_schedule = [
                    [6,-2,4,3,-5,-4,-3,5,2,-6],
                    [5,1,-3,-6,4,3,6,-4,-1,-5],
                    [-4,5,2,-1,6,-2,1,-6,-5,4],
                    [3,6,-1,-5,-2,1,5,2,-6,-3],
                    [-2,-3,6,4,1,-6,-4,-1,3,2],
                    [1,-4,-5,2,-3,5,-2,3,4,1]]

# Calculates the total distance cost of a schedule.
# Return negative value if the given schedule violates L/U constraints.
def calculate_distances(schedule, distances) -> int:
    no_teams = len(schedule)
    no_rounds = len(schedule[0])
    
    total_cost = 0
    for team in range(no_teams):
        current_location = team
        
        #A flag used to find violations in the lower/upper bounds.
        #Counts upwards per home game and downwards per away game, reseting to 0 every shift between 2 modes.
        travel_flag = 0
        
        for round in range(no_rounds):
            opponent_match = schedule[team][round]
            
            #Home game. Return home.
            if opponent_match > 0:
                
                #Case 1: Attempt to break Away streak while not having at least L away games.
                if travel_flag < 0 and abs(travel_flag) < L:
                    return -1
                if travel_flag < 0:
                    travel_flag = 0
                travel_flag += 1
                
                total_cost += distances[current_location][team]
                current_location = team
            #Away game. Move to them.
            else:
                #Case 2: Attempt to break Home streak while not having at least L home games.
                if travel_flag > 0 and travel_flag < L:
                    return -1
                if travel_flag > 0:
                    travel_flag = 0
                travel_flag -= 1
                
                opponent_index = abs(opponent_match)-1
                total_cost += distances[current_location][opponent_index]
                current_location = opponent_index
                
            #Case 3: Attempt to continue streak above upper limit U.
            if abs(travel_flag) > U:
                return -1  
        # Finally, if we end our match on an Away game, we have to return the team home.
        if schedule[team][-1] < 0:
            total_cost += distances[team][abs(schedule[team][-1])-1] 
    return total_cost


# Calculates the total cost of a schedule, regardless of feasibility as well as a bool of whether it is feasible
# If the schedule is feasible, it works exactly like calculate_distance.
# If it is not, it produces the square root of cost^2 + W*(1 + sqrt(v)ln(v/2)) where V is the number of violations.
# and W is a predefined weight.
WEIGHT = 1
def calculate_cost(schedule, distances) -> int:
    no_teams = len(schedule)
    no_rounds = len(schedule[0])
    
    total_cost = 0
    violations = 0
    for team in range(no_teams):
        current_location = team
        
        #A flag used to find violations in the lower/upper bounds.
        #Counts upwards per home game and downwards per away game, reseting to 0 every shift between 2 modes.
        travel_flag = 0
        
        for round in range(no_rounds):
            opponent_match = schedule[team][round]
            
            #Home game. Return home.
            if opponent_match > 0:
                
                #Case 1: Attempt to break Away streak while not having at least L away games.
                if travel_flag < 0 and abs(travel_flag) < L:
                    violations += 1
                if travel_flag < 0:
                    travel_flag = 0
                travel_flag += 1
                
                total_cost += distances[current_location][team]
                current_location = team
            #Away game. Move to them.
            else:
                #Case 2: Attempt to break Home streak while not having at least L home games.
                if travel_flag > 0 and travel_flag < L:
                    violations += 1
                if travel_flag > 0:
                    travel_flag = 0
                travel_flag -= 1
                
                opponent_index = abs(opponent_match)-1
                total_cost += distances[current_location][opponent_index]
                current_location = opponent_index
                
            #Case 3: Attempt to continue streak above upper limit U.
            if abs(travel_flag) > U:
                violations += 1

        # Finally, if we end our match on an Away game, we have to return the team home.
        if schedule[team][-1] < 0:
            total_cost += distances[team][abs(schedule[team][-1])-1]
            
    if violations == 0:  
        return (total_cost, 1)
    else:
        return (math.sqrt( total_cost**2 + WEIGHT*(1 + math.sqrt(violations)*math.log(violations/2))**2 ) , 0)


In [4]:
#----- Scheduling Moves / Neighborhood Shuffle -------

# Selects two random teams from the current schedule and swap the venues of the matches they host for each other.
# For example, if Team A and Team B play at Team A’s home in Round 1 and at Team B’s home in Round 2,
# after the swap, Round 1 will be hosted by Team B and Round 2 by Team A.
def swap_random_homes(schedule):
    no_teams = len(schedule)
    
    team_A = random.randrange(0, no_teams)
    team_B = random.randrange(0, no_teams)
    while team_B == team_A:
        team_B = random.randrange(0, no_teams)
    
    return swap_homes(schedule, team_A, team_B)
        
def swap_homes(schedule, team_A, team_B):
    no_rounds = len(schedule[0])
    
    for i in range(no_rounds):
        if abs(schedule[team_A][i])-1 == team_B:
            schedule[team_A][i] *= -1

        if abs(schedule[team_B][i])-1 == team_A:
            schedule[team_B][i] *= -1
    return schedule       
    

# Selects two random rounds in the provided schedule, and swaps them.
def swap_random_rounds(schedule):
    no_rounds = len(schedule[0])
    
    round_A = random.randrange(0, no_rounds)
    round_B = random.randrange(0, no_rounds)
    while round_A == round_B:
        round_B = random.randrange(0, no_rounds)
    
    return swap_rounds(schedule, round_A, round_B)

def swap_rounds(schedule, round_A, round_B):
    no_teams = len(schedule)
    for team in range(no_teams):
        schedule[team][round_A], schedule[team][round_B] = schedule[team][round_B], schedule[team][round_A]
    return schedule


# Selects two random teams from the provided schedule and swaps their schedules (except for the ones where they play with each other).
# Proceeds to fix accordingly the schedule of other teams to match the changes.
# For every round changed, there will be 4 matches changed (2 for the chosen teams, and 2 for their original parteners).
def swap_random_teams(schedule):
    no_teams = len(schedule)
    
    team_A = random.randrange(0, no_teams)
    team_B = random.randrange(0, no_teams)
    while team_B == team_A:
        team_B = random.randrange(0, no_teams)
        
    return swap_teams(schedule, team_A, team_B)
    
def swap_teams(schedule, team_A, team_B):
    no_rounds = len(schedule[0])
    
    for round in range(no_rounds):
        #Ignore matches played with each other.
        if abs(schedule[team_A][round])-1 == team_B or abs(schedule[team_B][round])-1 == team_A:
            continue
        
        original_team_A = schedule[team_A][round]
        original_team_B = schedule[team_B][round]
        
        #Swap matches.
        schedule[team_A][round] = original_team_B
        schedule[team_B][round] = original_team_A
        
        #Find the indices of who played with team A and B originally
        prev_team_A_opponent_index = abs(original_team_A)-1
        prev_team_B_opponent_index = abs(original_team_B)-1
        
        #Find whether the new games are at home or not.
        is_home_game_A = 1 if schedule[team_A][round] > 0 else -1
        is_home_game_B = 1 if schedule[team_B][round] > 0 else -1
        
        #Change original opponents to match swapped schedule.
        schedule[prev_team_A_opponent_index][round] = (team_B+1)*-is_home_game_B
        schedule[prev_team_B_opponent_index][round] = (team_A+1)*-is_home_game_A
    return schedule
        

# Given a team and 2 rounds on a provided schedule, it swaps those 2 rounds.
# Since this breaks the round-robin, we also swap those 2 rounds for all other "affected" teams.
# Assume a graph where the teams are Vertices and the Edges are matches between them on both rounds A and B. "Affected" teams are
# considered any team that belongs to the connected component of the provided team.
def swap_partial_rounds_for_team(schedule, team, round_A, round_B):
    swap_teams = []
    
    #Explore BFS style the graph
    queue = [team]
    while len(queue) > 0:
        currTeam = queue.pop()
        
        #Any explored team not found before is added to the list
        if currTeam in swap_teams:
            continue
        swap_teams.append(currTeam)
        
        #Check each round's match for connected nodes (opposing teams).
        #If not seen before, add them to the queue.
        match_A_team_index = abs(schedule[currTeam][round_A])-1
        if match_A_team_index not in swap_teams:
            queue.append(match_A_team_index)
            
        match_B_team_index = abs(schedule[currTeam][round_B])-1
        if match_B_team_index not in swap_teams:
            queue.append(match_B_team_index)
        
    #Having found all the teams in the connected component, we swap their matches on the two rounds.
    for t in swap_teams:
        schedule[t][round_A], schedule[t][round_B] = schedule[t][round_B], schedule[t][round_A]
    return schedule

# Given a schedule, a specific round and 2 teams, it swaps the match for that round between the 2 teams.
# The schedule is heavily rebalanced afterwards to make sure the round-robin is maintained.
# NOTE: This method is experimental. The paper does not describe it well enough for me to fully understand it.
# NOTE: This method produces illegal methods.
def swap_partial_teams_for_round(schedule, round, team_A, team_B):
    
    schedule[team_A][round], schedule[team_B][round] = schedule[team_B][round], schedule[team_A][round]
    
    no_teams = len(schedule)
    no_rounds = len(schedule[0])
    
    hasSwapped = True
    swapped_indices = [round]
    last_swap_item = schedule[team_A][round]
    
    # Step 1: Fix round-robin for the 2 teams by swapping matches between them until it's restored.
    while hasSwapped:
        hasSwapped = False
        for r in range(no_rounds):
            if r in swapped_indices:
                continue
            if schedule[team_A][r] == last_swap_item:
                schedule[team_A][r], schedule[team_B][r] = schedule[team_B][r], schedule[team_A][r]
                last_swap_item = schedule[team_A][r]
                swapped_indices.append(r)
                hasSwapped = True
                break
            
    # Step 2: In every Round we swapped the two teams together, we need to inform the other teams about this swap.
    # We simply need to changed those that played with team A to team B and vice versa.
    for t in range(no_teams):
        if t == team_A or t == team_B:
            continue
        
        for r in swapped_indices:
            if abs(schedule[t][r])-1 == team_A:
                m = 1 if schedule[t][r] > 0 else -1
                schedule[t][r] = (team_B+1)*m
            elif abs(schedule[t][r])-1 == team_B:
                m = 1 if schedule[t][r] > 0 else -1
                schedule[t][r] = (team_A+1)*m
    return schedule


In [5]:
# ------------ Produce All Moves for Subroutines -----------------

def swap_homes_all(schedule):
    neighbors = []
    no_teams = len(schedule)
    for t1 in range(no_teams):
        for t2 in range(no_teams):
            if t1 == t2:
                continue
            neighbor = swap_homes(copy.deepcopy(schedule), t1, t2)
            neighbors.append(neighbor)
    return neighbors

def swap_rounds_all(schedule):
    neighbors = []
    no_rounds = len(schedule[0])
    for r1 in range(no_rounds):
        for r2 in range(no_rounds):
            if r1 == r2:
                continue
            neighbor = swap_rounds(copy.deepcopy(schedule), r1, r2)
            neighbors.append(neighbor)
    return neighbors

def swap_teams_all(schedule):
    neighbors = []
    no_teams = len(schedule)
    for t1 in range(no_teams):
        for t2 in range(no_teams):
            if t1 == t2:
                continue
            neighbor = swap_teams(copy.deepcopy(schedule), t1, t2)
            neighbors.append(neighbor)
    return neighbors

def swap_partial_rounds_for_team_all(schedule):
    neighbors = []
    no_teams = len(schedule)
    no_rounds = len(schedule[0])
    for team in range(no_teams):
        for r1 in range(no_rounds):
            for r2 in range(no_rounds):
                if r1 == r2:
                    continue
                neighbor = swap_partial_rounds_for_team(copy.deepcopy(schedule), team, r1, r2)
                neighbors.append(neighbor)
    return neighbors

def swap_partial_teams_for_round_all(schedule):
    neighbors = []
    no_teams = len(schedule)
    no_rounds = len(schedule[0])
    for round in range(no_rounds):
        for t1 in range(no_teams):
            for t2 in range(no_teams):
                if t1 == t2:
                    continue
                neighbor = swap_partial_teams_for_round(copy.deepcopy(schedule), round, t1, t2)
                neighbors.append(neighbor)
    return neighbors

In [ ]:
# ------------------- Simulated Annealing -----------------------
#NL4 Parameters + Random Initial Schedule
L = 0
U = 3
"""
d = [[0, 745, 665, 929],
     [745, 0, 80, 337],
     [665, 80, 0, 380],
     [929, 337, 380, 0]]

# 8276 Best -> Found 8276 in 2.2s
schedule_4 = [[2, 3, -2, -3, 4, -4],
                    [-1, -4, 1, 4, -3, 3],
                    [4, -1, -4, 1, 2, -2],
                    [-3, 2, 3, -2, -1, 1]]
"""
d = [
    [   0,  745,  665,  929,  605,  521],  # team 0
    [ 745,    0,   80,  337, 1090,  315],  # team 1
    [ 665,   80,    0,  380, 1020,  257],  # team 2
    [ 929,  337,  380,    0, 1380,  408],  # team 3
    [ 605, 1090, 1020, 1380,    0, 1010],  # team 4
    [ 521,  315,  257,  408, 1010,    0]   # team 5
]
# 23916 Best  -> Found 25561 in avg 18.4s
schedule_6 = [
    [  6, -2,  3, -4,  5, -6,  2, -3,  4, -5],  # Team 1
    [ -5,  1, -6,  2, -3,  4, -1,  6, -2,  3],  # Team 2
    [  4, -5,  1, -6,  2, -3,  5, -1,  6, -2],  # Team 3
    [ -3,  6, -2,  5, -1,  6, -4,  2, -5,  1],  # Team 4
    [  2, -3,  6, -1,  4, -2,  3, -6,  1, -4],  # Team 5
    [ -1,  4, -5,  3, -6,  1, -2,  5, -3,  2]   # Team 6
]


INIT_SCHEDULE = schedule_6

In [36]:
# ----------- Simulated Annealing with Tabu Search Algorithm -----------------
tabu_list = []

bestFeasibleSchedule = None
bfs_cost = math.inf
bestInfeasibleSchedule = None
bis_cost = math.inf

init_cost, init_feasibility = calculate_cost(INIT_SCHEDULE, d)
if init_feasibility == 1:
    bestFeasibleSchedule = copy.deepcopy(INIT_SCHEDULE)
    bfs_cost = init_cost
else:
    bestInfeasibleSchedule = copy.deepcopy(INIT_SCHEDULE)
    bis_cost = init_cost

current_schedule = copy.deepcopy(INIT_SCHEDULE)
iters = 0
MAX_ITERS = 1_000
no_teams = len(current_schedule)
no_rounds = len(current_schedule[0])
while iters < MAX_ITERS:
    iters += 1
    
    solutions = []
    # Step 1: Generate all neighboring solutions.
    solutions += swap_homes_all(current_schedule)
    solutions += swap_rounds_all(current_schedule)
    solutions += swap_teams_all(current_schedule)
    solutions += swap_partial_rounds_for_team_all(current_schedule)
    #solutions += swap_partial_teams_for_round_all(current_schedule)
    
    batchBestSolution = None
    batchBestCost = math.inf
    batchBestFeasibility = -1
    for solution in solutions:
        c, f = calculate_cost(solution, d)

        # Case 1: Solution is feasible and better globally. Ignore Tabu rule due to Aspiration.
        if f == 1 and c < bfs_cost:
            current_schedule = solution
            bfs_cost = c
            bestFeasibleSchedule = solution
            iter = 0
            tabu_list.clear()
        
        # Case 2: Solution has been visited before. Ignore it.    
        if solution in tabu_list:
            continue
        tabu_list.append(solution)
        
        # Case 3: If we have not found a better solution in this neighborhood, place it as the best so far.
        if c < batchBestCost:
            batchBestSolution = solution
            batchBestCost = c
            batchBestFeasibility = f
    
    # If our batch's best solution is feasible, and better than our best found solution. Update it.
    if batchBestFeasibility == 1:
        if batchBestCost < bfs_cost:
            bfs_cost = batchBestCost
            bestFeasibleSchedule = batchBestSolution
            tabu_list.clear()
            iter = 0
            current_schedule = batchBestSolution
    else:
        # If our batch's best solution is not feasible, but also better than our previous non-feasible solution, update it.
        if batchBestCost < bis_cost:
            bis_cost = batchBestCost
            bestInfeasibleSchedule = batchBestSolution
            current_schedule = batchBestSolution
            tabu_list.clear()
            
    
bestFeasibleSchedule, bfs_cost          

([[6, 4, -2, -4, 5, -6, 3, 2, -3, -5],
  [4, 3, 1, -3, -3, -3, 5, -1, -5, 3],
  [-5, -2, -6, 2, 2, 2, -1, 6, 1, -2],
  [-2, -5, -2, 5, -1, -3, -4, 2, 6, 1],
  [3, 1, 6, -1, 4, -2, -2, -6, 2, -4],
  [-1, -2, 3, 2, -6, 1, -2, -3, 4, 2]],
 25561)